<a href="https://colab.research.google.com/github/tw7366/Sentiment-Analysis-of-Movie-Reviews/blob/master/Sentiment%20Analysis%20of%20Movie%20Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import re

In [2]:
reviews_df=pd.read_csv('drive/My Drive/IMDB Dataset.csv')
reviews = np.array(reviews_df['review'])
sentiment = np.array(reviews_df['sentiment'])
print(reviews[:3])

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer

stops = set(stopwords.words("english"))
lemma = WordNetLemmatizer()

def preprocess_review(review_list):
    """
    param: a list of string
    return: a list of string
    preprocess reviews by changing all alphabets to lowercase, removing punctuations, and cleaning html leftover code
    """
    for i, review in enumerate(review_list):
        review = review.replace('<br /><br />', ' ')
        review = review.lower()
        review = re.sub(r"[^A-Za-z0-9' ]+", '', review)
        review = review.replace('  ', ' ')
        review_list[i] = review
    return review_list


def remove_stops(string_list):
  """
  param: a list of strings
  return: a list of strings without stopwords
  remove all stopwords from a list
  """
  # tokenized_list = []
  # final_list = []

  # tokenize words
  for i, string in enumerate(string_list):
    pre_filtered = word_tokenize(string)
    filtered = [lemma.lemmatize(word, pos = "v") for word in pre_filtered]
    filtered = [lemma.lemmatize(word, pos = "n") for word in filtered]
    filtered = [word for word in filtered if not word in stops]
    filtered_string = ' '.join(filtered)
    string_list[i] = filtered_string 
  return string_list


preprocessed_reviews = preprocess_review(reviews)
preprocessed_reviews = remove_stops(preprocessed_reviews)

In [4]:
# to divide train & test sets
test_sample_size = int(0.1*len(preprocessed_reviews))  # 10% of data as the validation set

# for sentiment
sentiment = [1 if x=='positive' else 0 for x in sentiment]

# separate data to train & test sets
X_test, X_train = (np.array(preprocessed_reviews[:test_sample_size]), 
                   np.array(preprocessed_reviews[test_sample_size:])
)

y_test, y_train = (np.array(sentiment[:test_sample_size]), 
                   np.array(sentiment[test_sample_size:])
)


tokenizer = Tokenizer(oov_token='<OOV>')  # for the unknown words
tokenizer.fit_on_texts(X_train)

vocab_count = len(tokenizer.word_index) + 1  # +1 is for padding

# create padded sequences
training_sequences = tokenizer.texts_to_sequences(X_train)  # tokenizer.word_index to see indexes
training_padded = pad_sequences(training_sequences, padding='post')  # pad sequences with 0s

testing_sequences = tokenizer.texts_to_sequences(X_test)  # tokenizer.word_index to see indexes
testing_padded = pad_sequences(testing_sequences, padding='post')  # pad sequences with 0s

input_length = len(testing_padded[0])  # length of all sequences


# build a model
model = keras.models.Sequential()
model.add(keras.layers.Embedding(input_dim=vocab_count,
                                 output_dim=4,
                                 input_length=input_length,
                                 mask_zero = True)
)
model.add(keras.layers.GlobalAveragePooling1D())  # find the average of vectors to get sentiment
model.add(keras.layers.Dense(64, activation='relu'))  # hidden layer
model.add(keras.layers.Dense(16, activation='relu'))  # hidden layer
model.add(keras.layers.Dense(1, activation='sigmoid'))  # output layer

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 947, 4)            547012    
_________________________________________________________________
global_average_pooling1d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 548,389
Trainable params: 548,389
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(training_padded, y_train, epochs=5, batch_size=512,
          validation_data=(testing_padded, y_test)
)

Epoch 1/5
88/88 [==============================] - 2s 27ms/step - loss: 0.6748 - accuracy: 0.6854 - val_loss: 0.6177 - val_accuracy: 0.8174
Epoch 2/5
88/88 [==============================] - 2s 25ms/step - loss: 0.4624 - accuracy: 0.8567 - val_loss: 0.3284 - val_accuracy: 0.8830
Epoch 3/5
88/88 [==============================] - 2s 25ms/step - loss: 0.2577 - accuracy: 0.9057 - val_loss: 0.2577 - val_accuracy: 0.8998
Epoch 4/5
88/88 [==============================] - 2s 25ms/step - loss: 0.1906 - accuracy: 0.9304 - val_loss: 0.2419 - val_accuracy: 0.9056
Epoch 5/5
88/88 [==============================] - 2s 25ms/step - loss: 0.1507 - accuracy: 0.9466 - val_loss: 0.2430 - val_accuracy: 0.9058


In [6]:
index_list = []
predictions = model.predict(training_padded)
predictions = ['negative' if x <= 0.5 else 'positive' for x in predictions]
y_test = ['negative' if x <= 0.5 else 'positive' for x in y_test]
for i, (prediction, truth) in enumerate(zip(predictions, y_test)):
    if prediction != truth:
        index_list.append(i)

print(f'The model has predicted {len(index_list)} wrong of {len(predictions)}')

The model has predicted 2498 wrong of 45000


In [7]:
for i, item in enumerate(index_list):
    if i <= 5:
        print(X_test[item])
        print(f'Prediction: {predictions[item]}')
        print(f'Sentiment: {y_test[item]}')
        print('============================')

one reviewer mention watch 1 oz episode 'll hook right exactly happen first thing strike oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home manyaryans muslim gangsta latino christian italian irish moreso scuffle death star dodgy deal shady agreement never far away would say main appeal show due fact go show would n't dare forget pretty picture paint mainstream audience forget charm forget romanceoz n't mess around first episode ever saw strike nasty surreal could n't say ready watch develop taste oz get accustom high level graphic violence violence injustice crook guard 'll sell nickel inmate 'll kill order get away well mannered middle class inmate turn prison bitch due lack street skill prison experienc

The model made good assumptions for most of them, but it got easily confused by words like 'good', 'bad', 'well', etc. 

LSTM Model

In [8]:
y_test = [0 if x == 'negative' else 1 for x in y_test]

# the model takes numpy arrays as inputs
y_test = np.asarray(y_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

for i, (train_item, test_item) in enumerate(zip(training_padded, testing_padded)):
    training_padded[i] = np.asarray(train_item).astype(np.float32)
    testing_padded[i] = np.asarray(test_item).astype(np.float32)

In [9]:
from tensorflow.keras.layers import SpatialDropout1D, Embedding, LSTM, Bidirectional, Dense

new_model = keras.models.Sequential()
new_model.add(Embedding(input_dim=vocab_count,
                        output_dim=4,
                        input_length=input_length))
new_model.add(Bidirectional(tf.keras.layers.LSTM(16, dropout=0.2)))
new_model.add(Dense(16, activation='relu'))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

print(new_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 947, 4)            547012    
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                2688      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 550,245
Trainable params: 550,245
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
new_model.fit(training_padded, y_train, epochs=5, batch_size=512,
              validation_data=(testing_padded, y_test)
)

Epoch 1/5
88/88 [==============================] - 13s 153ms/step - loss: 0.6449 - accuracy: 0.6418 - val_loss: 0.4477 - val_accuracy: 0.8022
Epoch 2/5
88/88 [==============================] - 13s 149ms/step - loss: 0.3356 - accuracy: 0.8669 - val_loss: 0.2662 - val_accuracy: 0.8988
Epoch 3/5
88/88 [==============================] - 13s 148ms/step - loss: 0.2073 - accuracy: 0.9228 - val_loss: 0.2419 - val_accuracy: 0.9024
Epoch 4/5
88/88 [==============================] - 13s 149ms/step - loss: 0.1553 - accuracy: 0.9454 - val_loss: 0.2503 - val_accuracy: 0.9020
Epoch 5/5
88/88 [==============================] - 13s 149ms/step - loss: 0.1218 - accuracy: 0.9573 - val_loss: 0.2651 - val_accuracy: 0.8990


In [11]:
new_index_list = []
new_predictions = new_model.predict(testing_padded)
new_predictions = ['negative' if x <= 0.5 else 'positive' for x in new_predictions]
y_test = ['negative' if x <= 0.5 else 'positive' for x in y_test]
for i, (prediction, truth) in enumerate(zip(new_predictions, y_test)):
    if prediction != truth:
        new_index_list.append(i)

print(f'The model has predicted {len(new_index_list)} wrong of {len(new_predictions)}')

The model has predicted 505 wrong of 5000
